## Flowbot inference

In [ ]:
ckpt_path = "/home/yishu/open_anything_diffusion/logs/train_synthetic_synthetic/2024-01-24/10-33-20/checkpoints/epoch=794-step=1590.ckpt"

In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf
initialize(config_path="../../configs", version_base="1.3")
cfg = compose(config_name="train_synthetic")

In [ ]:
from open_anything_diffusion.datasets.synthetic_dataset import SyntheticDataModule
datamodule = SyntheticDataModule(
    batch_size=cfg.training.batch_size,
    seed=cfg.seed
)
train_loader = datamodule.train_dataloader()
train_val_loader = datamodule.train_val_dataloader()
val_loader = datamodule.val_dataloader()
unseen_loader = datamodule.unseen_dataloader()

In [ ]:
import rpad.pyg.nets.pointnet2 as pnp
from open_anything_diffusion.models.flow_trajectory_predictor import (
    FlowTrajectoryTrainingModule,
)
network = pnp.PN2Dense(
    in_channels=0,
    out_channels=3,
    p=pnp.PN2DenseParams(),
)

model = FlowTrajectoryTrainingModule(network, cfg.training)

In [ ]:
import torch
ckpt = torch.load(ckpt_path)
model.load_state_dict(ckpt["state_dict"])

In [ ]:
samples = list(enumerate(train_val_loader))

In [ ]:
from open_anything_diffusion.metrics.trajectory import artflownet_loss, flow_metrics, normalize_trajectory
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
import numpy as np

@torch.no_grad()
def flowbot_visual(batch, model):  # 1 sample batch
    model.eval()
    
    animation = FlowNetAnimation()
    pcd = batch.pos.cpu().numpy()
    f_pred = model(batch)
    f_pred = normalize_trajectory(f_pred[:, None, :])

    animation.add_trace(
        torch.as_tensor(pcd),
        # torch.as_tensor([pcd[mask]]),
        # torch.as_tensor([flow[mask]]),
        torch.as_tensor([pcd]),
        torch.as_tensor([f_pred.squeeze().cpu().numpy()]),
        "red",
    )

    # largest_mag: float = torch.linalg.norm(
    #     f_pred, ord=2, dim=-1
    # ).max()
    # f_pred = f_pred / (largest_mag + 1e-6)

    # Compute the loss.
    n_nodes = torch.as_tensor([d.num_nodes for d in batch.to_data_list()]).to("cuda")  # type: ignore
    f_ix = batch.mask.bool()
    f_target = batch.delta
    f_target = normalize_trajectory(f_target)

    f_target = f_target.float()
    loss = artflownet_loss(f_pred, f_target, n_nodes)

    # Compute some metrics on flow-only regions.
    rmse, cos_dist, mag_error = flow_metrics(
        f_pred[f_ix], batch.delta[f_ix]
    )

    return rmse, cos_dist, mag_error, loss, animation

In [ ]:
all_rmse = 0
all_cos_dist = 0
all_mag_error = 0
all_loss = 0
model = model.cuda()
for i in range(len(samples)):
    sample = samples[i][1].cuda()
    batch = sample
    rmse, cos_dist, mag_error, loss, animation = flowbot_visual(sample, model)
    all_rmse += rmse.item()
    all_cos_dist += cos_dist.item()
    all_loss += loss.item()
    all_mag_error += mag_error.item()

all_rmse /= len(samples)
all_cos_dist /= len(samples)
all_mag_error /= len(samples)
all_loss /= len(samples)
print(f"rmse:{all_rmse:.4f}, cos:{all_cos_dist:.4f}, mag:{all_mag_error:.4f}, flowloss:{all_loss:.4f}")

### Example 1

In [ ]:
len(samples)

In [ ]:
sample = samples[0][1].cuda()
batch = sample
model = model.cuda()

In [ ]:
rmse, cos_dist, mag_error, loss, animation = flowbot_visual(sample, model)
print(f"rmse:{rmse:.4f}, cos:{cos_dist:.4f}, mag:{mag_error:.4f}, flowloss:{loss:.4f}")
fig = animation.animate()
fig.show()

In [ ]:
print(rmse, cos_dist, mag_error, loss)

###  Example 2

In [ ]:
sample = samples[1][1].cuda()
batch = sample
model = model.cuda()

In [ ]:
rmse, cos_dist, mag_error, loss, animation = flowbot_visual(sample, model)
fig = animation.animate()
fig.show()

In [ ]:
print(rmse, cos_dist, mag_error, loss)

### Test 1

In [ ]:
sample = samples[0][1].cuda()
batch = sample
model = model.cuda()

rmse, cos_dist, mag_error, loss, animation = flowbot_visual(sample, model)
fig = animation.animate()
fig.show()

In [ ]:
print(rmse, cos_dist, mag_error, loss)